#### Run this code for more epochs for better result

In [10]:
####### Installing the necessary library
!pip install wandb
!pip install simpletransformers
!pip show simpletransformers
!pip install --upgrade simpletransformers

Name: simpletransformers
Version: 0.64.3
Summary: An easy-to-use wrapper library for the Transformers library.
Home-page: https://github.com/ThilinaRajapakse/simpletransformers/
Author: Thilina Rajapakse
Author-email: chaturangarajapakshe@gmail.com
License: 
Location: /home/sugam/miniconda3/envs/simple_transformer/lib/python3.9/site-packages
Requires: datasets, numpy, pandas, regex, requests, scikit-learn, scipy, sentencepiece, seqeval, streamlit, tensorboard, tokenizers, tqdm, transformers, wandb
Required-by: 


In [2]:
# The directory for top level folder
dir_ = "/home/sugam/Work/10-19 NLP/12 Projects/Resume Builder/data/"

processed_data_dir = dir_+'Processed/'

In [3]:
# Importing the necessary library
import json # for loading dataset
import numpy as np
import logging
import torch
import wandb
import torch
import logging
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

# from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

2023-10-30 11:09:08.335830: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/sugam/miniconda3/envs/simple_transformer/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

with open(processed_data_dir+"train_json.json") as file:
  train_data = json.load(file)

with open(processed_data_dir+"valid_json.json") as file:
  valid_data = json.load(file)

with open(processed_data_dir+"test_json.json") as file:
  test_data = json.load(file)

In [5]:
test_data[0]

{'context': 'JT_MASTER TRAINER-RETAIL,CO_Lok Bharti Skilling Solution Pvt Ltd,JR_Teacher, Private Tutor Keyskills bfsi documentation audit ttt good presentation skills retail sales retail training verbal communication marketing Desired Candidate Profile ,JL_Surat,ED_BTech,BE-Mechanical,EX_2 - 4 yrs,SK_Teaching,IN_IT-Software , Software Services,PR_Not Disclosed by Recruiter',
 'qas': [{'id': '100001',
   'is_impossible': False,
   'question': 'What is job provided by Lok Bharti Skilling Solution Pvt Ltd company?',
   'answers': [{'text': 'MASTER TRAINER-RETAIL', 'answer_start': 3}]},
  {'id': '100002',
   'is_impossible': False,
   'question': 'What is salary provided by Lok Bharti Skilling Solution Pvt Ltd company for MASTER TRAINER-RETAIL post?',
   'answers': [{'text': 'Not Disclosed by Recruiter', 'answer_start': 337}]},
  {'id': '100003',
   'is_impossible': False,
   'question': 'What are skills required for MASTER TRAINER-RETAIL post in Lok Bharti Skilling Solution Pvt Ltd compa

In [5]:
# Checking if CUDA is available or not ?
cuda_available = torch.cuda.is_available()
cuda_available

False

In [ ]:
model_type="bert"
model_name= "bert-base-cased"

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Run this if this notebook requires re run
# It is mandatory to clear the output dir before re running the simple transformer model without removal it will not work
!rm -r outputs/*

train_args = {
    'learning_rate': 5e-6,
    "use_cached_eval_features": True,
    "output_dir": f"/content/drive/MyDrive/NLP Project/outputs/{model_type}",
    "best_model_dir": f"/content/drive/MyDrive/NLP Project/outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    'weight_decay': 3,
    'warmup_steps': 5,
    'max_grad_norm': 5.0,
    "wandb_project": "NLP",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    'num_train_epochs': 1,
    'max_seq_length': 384,
    'doc_stride': 128,
    'overwrite_output_dir': True,
    'reprocess_input_data': False,
    "save_eval_checkpoints": False,
    'train_batch_size': 6,
    #"eval_batch_size": 64,
    'gradient_accumulation_steps': 8,
    "n_best_size":5,
    "use_multiprocessing": True,
}

# model = QuestionAnsweringModel('bert', 'bert-base-uncased', args=train_args)
model = QuestionAnsweringModel(model_type, model_name, args=train_args)


In [ ]:
model.train_model(train_data,
                  eval_data = valid_data,
                  use_cuda = cuda_available)

In [ ]:
result, model_outputs, wrong_preds = model.eval_model(valid_data)

In [ ]:
result, model_outputs, wrong_preds = model.eval_model(test_data)